In [25]:
from dace.spectroscopy import Spectroscopy
import dace as Dace
import etta
from astropy.coordinates import SkyCoord
import pandas as pd
from astropy import units as u
from astropy.coordinates import Angle

In [9]:
def get_tess_toi_df():
    tess_toi_df = etta.download_toi(sort = 'toi')
    print('data fetched!')
    tess_toi_df = tess_toi_df.sort_index(ascending=False) #puts the most recent TOIs first
    tess_toi_df_rows = tess_toi_df.shape[0]
    new_toi = tess_toi_df.first_valid_index()
    new_toi_date = tess_toi_df.iloc[0]['Date Modified']
    new_toi_comments = tess_toi_df.iloc[0]['Comments']
    print("\n")
    print(f'TESS TOI table has {tess_toi_df_rows} TOIs. The most recent is {new_toi} which was updated on {new_toi_date} with the notes: {new_toi_comments}.')

    return(tess_toi_df)

def get_coords(tess_toi_df, toi):
    """gets the RA & Dec of the target to be able to do a cone search."""
    toi_ra = tess_toi_df.loc[toi].RA
    toi_dec = tess_toi_df.loc[toi].Dec
    toi_coords = SkyCoord(ra = toi_ra, dec = toi_dec, unit = (u.hour, u.degree), frame='icrs')

    return toi_coords

In [11]:
exofop_table = get_tess_toi_df()

Fetching data from https://exofop.ipac.caltech.edu/tess/download_toi.php?output=pipe&sort=toi
data fetched!


TESS TOI table has 4704 TOIs. The most recent is 4644.01 which was updated on 2021-11-25 12:05:30 with the notes: small a/Rs; possibly variability.


In [20]:
coords = get_coords(exofop_table, exofop_table.index)

In [60]:
coords

<SkyCoord (ICRS): (ra, dec) in deg
    [( 41.728875  ,  10.025975  ), ( 32.21795833,   4.30692778),
     ( 30.88466667,   6.81586944), ..., (312.4575    , -24.42869444),
     ( 87.13983333, -63.98832778), (318.737     , -55.87186389)]>

In [99]:
table = Spectroscopy.query_region(coords[29], Angle('25d'), output_format='pandas',filters={'ins_name': {'contains': 'HARPS'}})

,obj_id_catname,obj_pos_coordinates_hms_dms,ins_name,prog_id,obj_date_bjd,date_night,ins_drs_version,pub_ref,pub_bibcode,public
0,Gl176,04:42:55.78 / 18:57:29.4,HARPS03,072.C-0488,54117.631291,2007-01-16,3.5,,,True
1,Gl273,07:27:24.5 / 05:13:32.8,HARPS03,072.C-0488,54552.534316,2008-03-26,3.5,,,True
2,Gl176,04:42:55.78 / 18:57:29.4,HARPS03,072.C-0488,53372.672289,2005-01-01-3.5,3.5,,,True
3,Gl176,04:42:55.78 / 18:57:29.4,HARPS03,072.C-0488,54428.729841,2007-11-23,3.5,,,True
4,Gl176,04:42:55.78 / 18:57:29.4,HARPS03,072.C-0488,53812.506114,2006-03-17,3.5,,,True
...,...,...,...,...,...,...,...,...,...,...
104,Gl176,04:42:55.78 / 18:57:29.4,HARPS03,072.C-0488,53811.510284,2006-03-16,3.5,,,True
105,Gl176,04:42:55.78 / 18:57:29.4,HARPS03,072.C-0488,53695.679077,2005-11-20,3.5,,,True
106,Gl273,07:27:24.5 / 05:13:32.8,HARPS03,072.C-0488,52986.769625,2003-12-12,3.5,,,True
107,Gl176,04:42:55.78 / 18:57:29.4,HARPS03,072.C-0488,54170.501820,2007-03-10,3.5,,,True


In [134]:
filepath = table[['file_rootpath']].loc[0]
filepath = filepath.item()
filename = filepath.split('/')[-1]

In [138]:
Spectroscopy.download_files('all', filename, '/home/z5345592/scripts/')

2021-12-02 15:52:34,794 - INFO - Downloading file on location : /home/z5345592/scripts//spectroDownload.tar.gz


 Download : 752 MB

KeyboardInterrupt: 